# 日本語テキストデータの分類

日本語ニュースフィードのカテゴリを教師データとし、カテゴリを予測

1. ニュースフィードから教師データを作成
2. 日本語テキストの前処理、ベクトル化
3. 分類モデルの作成（学習）
4. 精度の検証

In [8]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import re
import spacy
from sklearn.feature_extraction.text import CountVectorizer

# matplotlib: 日本語フォントの設定
from matplotlib import rcParams
rcParams['font.family'] = 'sans-serif'
rcParams['font.sans-serif'] = ['Hiragino Maru Gothic Pro', 'Yu Gothic', 'Meirio', 
                               'Takao', 'IPAexGothic', 'IPAPGothic', 'Noto Sans CJK JP']

# 日本語モデル
nlp = spacy.load('ja_core_news_lg')

# フィードデータの読み込み、確認
feeds = pd.read_csv('data/output_jp.csv')

# title と summary を結合
# str.cat() により複数列の文字列を結合
# - sep=' ': 間に挟む文字列
# - na_rep='': NaN は空文字列に変換（指定しないと結合結果が NaN になる）
feeds['text'] = feeds['title'].str.cat(feeds['summary'], sep='。', na_rep='')

# 不要になった列を削除した処理用の DataFrame
df = feeds.drop(['title', 'summary'], axis=1)

# 確認
df.head()

,url,text
0,https://www.nhk.or.jp/rss/news/cat0.xml,大阪 クリニック放火事件からまもなく1年 遺族が手記公開。大阪のビルでクリニックが放火され、...
1,https://www.nhk.or.jp/rss/news/cat0.xml,ワールドカップ 日本 森保監督 今大会の成果と今後の課題は。サッカーのワールドカップカタール...
2,https://www.nhk.or.jp/rss/news/cat0.xml,ロシア空軍基地に“ウクライナ軍の無人機攻撃” 大きな打撃か。ロシア国内の複数の空軍基地で爆発...
3,https://www.nhk.or.jp/rss/news/cat0.xml,サッカー日本代表 たどり着いたもう1つの“新しい景色”。11月20日に開幕したサッカーのワー...
4,https://www.nhk.or.jp/rss/news/cat0.xml,北朝鮮 日本海向け約100発砲撃 2日連続 “米韓両軍への警告”。韓国軍は、北朝鮮が6日、日...


### 教師データの作成

URLに基づいた教師ラベルの設定：

label 0
- business
- economy
- politics
- cat4 (nhk)
- cat5 (nhk)

label 1
- culture
- science
- sport
- cat2 (nhk)
- cat7 (nhk)

label 2
- 上記以外

In [9]:
label_0 = ['business', 'economy', 'politics', 'cat4', 'cat5']
label_1 = ['culture', 'science', 'sport', 'cat2', 'cat7']

# label_0 の単語が url の中に含まれている：0
# label_1 の単語が url の中に含まれている：1
# それ以外：2
def get_label(url):
    url = url.lower()
    # map: label_0 の単語それぞれについて、url の中に含まれているか真偽を返す
    # any: mapの結果について論理和をとる
    if any(map(lambda x: x in url, label_0)):
        return 0
    if any(map(lambda x: x in url, label_1)):
        return 1
    else:
        return 2

# df['url'] について get_label を適用した結果を df['label'] として追加
df['label'] = df['url'].map(lambda x: get_label(x))
# 各ラベルの数を確認
df['label'].value_counts()

2    2087
0     766
Name: label, dtype: int64

In [10]:
# label 0, 1 を取り出す
df = df.query('label != 2')
# 数を確認
df['label'].value_counts()

0    766
Name: label, dtype: int64

### 日本語テキストに対する前処理

- 表記の正規化
- トークン化（形態素解析）
- ストップワードの除去
- 見出し語化

In [11]:
# 不要な単語を除去
# - ストップワード (is_stop)
# - いくつかの品詞
#     AUX: 助動詞
#     PUNCT: 句読点
#     SPACE: 空白文字
#     SYM: 記号
#     X: その他
# - うまく取り除けない単語や文字
stop_pos = ['AUX', 'PUNCT', 'SPACE', 'SYM', 'X']
stop_words = ['.']

def token_to_add(w):
    t = w.text    # 単語
    p = w.pos_    # 品詞
    l = w.lemma_  # 原型

    # ストップワードは None を返す
    if w.is_stop:
        return None
    if p in stop_pos:
        return None
    if l in stop_words:
        return None

    if len(l) == 0:
        return t
    return l

def preprocess(text):
    tokens = []
    
    for w in nlp(text):
        t = token_to_add(w)
        if t is not None:
            tokens.append(t)

    # トークンのリストを返す
    return tokens

### テキストのベクトル化

- Bag of Words (BoW)

In [12]:
# 初期化
vectorizer = CountVectorizer(tokenizer=preprocess)
# ベクトル化
vector = vectorizer.fit_transform(df.text)

### 分類モデルの作成（学習）

- ナイーブベイズ分類器

In [13]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix
from sklearn.metrics import roc_curve, auc
from sklearn.naive_bayes import MultinomialNB

# 説明変数、目的変数
# - vector が sparse のため toarray() により dense に変換
X = vector.toarray()
Y = df.label

# ナイーブベイズ分類器
# - 特徴量（説明変数）は整数のカウントデータ（単語の出現頻度など）
model = MultinomialNB()

# 学習
model.fit(X, Y)
# 教師データについてモデルからの予測値を計算
Y_predict = model.predict(X)

# 特異度の計算
matrix = confusion_matrix(Y, Y_predict)
specificity = matrix[0, 0] / (matrix[0, 1] + matrix[0, 0]) # TODO: エラーになった
# 精度
print('正確度: {:.3f}, 適合度: {:.3f}, 再現率: {:.3f}, 特異度: {:.3f}'.format(
    accuracy_score(Y, Y_predict), precision_score(Y, Y_predict),
    recall_score(Y, Y_predict), specificity))
# ROC, AUC
Y_proba = model.predict_proba(X)
fpr, tpr, thresholds = roc_curve(Y, Y_proba[:, 1])
plt.plot(fpr, tpr, label='AUC={:.3f}'.format(auc(fpr, tpr)))
plt.xlabel('偽陽性率 (FP率)')
plt.ylabel('真陽性率 (TP率)')
plt.title('ROC曲線')
plt.legend()
plt.show()

IndexError: index 1 is out of bounds for axis 1 with size 1

### 交差検証法により予測精度を検証

- ナイーブベイズ分類器

In [14]:
from sklearn.model_selection import cross_val_score

# 交差検証の実行
score = cross_val_score(model, X, Y, cv=10, scoring='roc_auc')
print('AUC={:.3f} (+/- {:.3f})'.format(score.mean(), score.std()))

AUC=nan (+/- nan)


/Users/kazuya/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/kazuya/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 358, in _score
    y_pred = method_caller(clf, "decision_function", X)
  File "/Users/kazuya/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 71, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'MultinomialNB' object has no attribute 'decision_function'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/kazuya/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/Users/kazuya/opt/anaconda3/lib/python3.9/s

### その他の分類器

- ロジスティック回帰
- 決定木
- ランダムフォレスト
- SVM

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

models = [
    LogisticRegression(),
    DecisionTreeClassifier(),
    MultinomialNB(),
    RandomForestClassifier(),
    SVC()
]

for model in models:
    # 交差検証の実行
    score = cross_val_score(model, X, Y, cv=10, scoring='roc_auc')
    print('{}: AUC={:.3f} (+/- {:.3f})'.format(model, score.mean(), score.std()))